In [1]:
## Grapgh obtained from FG from build graph

def path_length(some_path):
    wgs84 = pyproj.Geod(ellps='WGS84')
    some_path_dist = 0
    for node in enumerate(some_path):
        orig = nx.get_node_attributes(FG, "geometry")[some_path[node[0]]]
        dest = nx.get_node_attributes(FG, "geometry")[some_path[node[0] + 1]]
        some_path_dist += wgs84.inv(shapely.geometry.asShape(orig).x, shapely.geometry.asShape(orig).y, 
                                         shapely.geometry.asShape(dest).x, shapely.geometry.asShape(dest).y)[2]
        if node[0] + 2 == len(some_path):
                    break
    return some_path_dist

def get_restrictions(path, graph):
    """
    Pickup all restrictions for a given path
    Restrictions are put on the nodes when creating the graph. 
    Now loop over the nodes and check if they have a restriction.
    Restrictions like height or closed nodes but others are possible too.
    """
    restrictions = {'closed':False, 'height': 99}
    height_restriction_on_graph = nx.get_node_attributes(graph, 'height_restriction')
    closed_nodes = nx.get_node_attributes(graph, 'closed')
    for node in path:
        rest = height_restriction_on_graph[node]
        if rest < restrictions['height']:
            restrictions['height'] = rest
        if closed_nodes[node]:
            restrictions['closed'] = True
    return restrictions    
    
class NodePath():
    """
    This object contains some more info about the path
    like: restrictions, length in m etc. 
    Can be used later when selecting proper path for a vessel.
    """
    def __init__(self, path, graph, indentifier):
        self.nodes = path
        self.length = path_length(path)
        self.name = "Path_{}".format(indentifier)
        # get restrictions for the nodes
        restrictions = get_restrictions(path, graph)
        self.max_height = restrictions['height']
        self.closed = restrictions['closed']
        
    def __type__(self):
        return "node_path" 
    
    def __str__(self):
        return "from node {} to  node {}, covering distance: {}, closed:{}, max height: {}".format(self.nodes[-1], self.nodes[0] ,self.length, self.closed, self.max_height)
    def plot(self, graph, save=False, filename=''):
        """
        Plot the path
        """
        locks = [178, 179, 220, 221, 155, 156]
        #new
        bridges = [340,103]
        for key in dict(graph.nodes):
            graph.nodes[key]["Path"] = 0.5
            graph.nodes[key]["Color"] = "Red"
            for i, node in enumerate(self.nodes):
                if key == node:
                    if i == 0:
                        graph.nodes[key]["Path"] = 200
                        graph.nodes[key]["Color"] = "Yellow"
                    elif i == len(self.nodes) - 1:
                        graph.nodes[key]["Path"] = 200
                        graph.nodes[key]["Color"] = "Blue"
                    elif node in locks:
                        # here we give other color to the locks
                        graph.nodes[key]["Path"] = 200
                        graph.nodes[key]["Color"] = "Red"
                    elif node in bridges:
                        graph.nodes[key]["Path"] = 200
                        graph.nodes[key]["Color"] = "Orange"
                    else:
                        graph.nodes[key]["Path"] = 50
                        graph.nodes[key]["Color"] = "Green"
                        break

        plt.figure(figsize=(18,18))
        nx.draw(graph, nx.get_node_attributes(graph, 'pos'), 
                with_labels = True, 
                node_size = list(dict(nx.get_node_attributes(graph, 'Path')).values()), 
                font_size = 3, 
                node_color = list(dict(nx.get_node_attributes(graph, 'Color')).values()), 
                width = 0.4, 
                arrowsize = 10)
        if (save):
            plt.savefig('img/' + filename + '.png')
        plt.show()

# Get all simple paths (which are lists of nodes)
# Take the list of nodes and put it in a NodePath object as shown above
# This object contains some more info about the path
# like: restrictions, length in m etc. 

paths = []
i = 1
for path in nx.all_simple_paths(FG_S, 263, 41): # simplepaths geeft lijst met node nummers
    #paths.append(NodePath(path, FG_S, i)) # maak van ieder lijst met nodes een NodePath obect en stop het in lijst
    #i+=1
    if i == 2:
        paths.append(NodePath(path, FG_S, "Scheldt-Rhine"))
        
    elif i == 15:
        paths.append(NodePath(path, FG_S, "Zuid-Beveland"))
     #new for bathseroute !!!  
    elif i == 5:
        paths.append(NodePath(path, FG_S, "Bathse-canal"))
        
    else: paths.append(NodePath(path, FG_S, i)) # maak van ieder lijst met nodes een NodePath obect en stop het in lijst
    i+=1
        
    
# this returns al the node paths sorted on length (shortest path first)
ordered_paths = sorted(paths, key=lambda paths: paths.length)

NameError: name 'nx' is not defined